<a href="https://colab.research.google.com/github/peppertaco/AI/blob/main/llama7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#우바부가 API + SillyLossy 확장기능 모딩 태번

In [ ]:
#@title Tap this if you play on Mobile { display-mode: "form" }
#Taken from KoboldAI colab
%%html
<b>Press play on the music player to keep the tab alive, then start TavernAI below (Uses only 13MB of data)</b><br/>
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [ ]:
#@title <b><font color = "red">TavernAI</b>
#@markdown <font color = "red"><-- 눌러 (≖ ‸ ≖ ✿)

use_google_drive = True #@param {type:"boolean"}
extras_enable_captioning = True #@param {type:"boolean"}
Captions_Model = "Salesforce/blip-image-captioning-large" #@param [ "Salesforce/blip-image-captioning-large", "Salesforce/blip-image-captioning-base" ]
#@markdown * Salesforce/blip-image-captioning-large - good base model
#@markdown * Salesforce/blip-image-captioning-base - slightly faster but less accurate
extras_enable_emotions = True #@param {type:"boolean"}
Emotions_Model = "bhadresh-savani/distilbert-base-uncased-emotion" #@param ["bhadresh-savani/distilbert-base-uncased-emotion", "joeddav/distilbert-base-uncased-go-emotions-student"]
#@markdown * bhadresh-savani/distilbert-base-uncased-emotion = 6 supported emotions<br>
#@markdown * joeddav/distilbert-base-uncased-go-emotions-student = 28 supported emotions
extras_enable_memory = True #@param {type:"boolean"}
Memory_Model = "Qiliang/bart-large-cnn-samsum-ChatGPT_v3" #@param [ "Qiliang/bart-large-cnn-samsum-ChatGPT_v3", "Qiliang/bart-large-cnn-samsum-ElectrifAi_v10", "distilbart-xsum-12-3" ]
#@markdown * Qiliang/bart-large-cnn-samsum-ChatGPT_v3 - summarization model optimized for chats
#@markdown * Qiliang/bart-large-cnn-samsum-ElectrifAi_v10 - nice results so far, but still being evaluated
#@markdown * distilbart-xsum-12-3 - faster, but pretty basic alternative

!nvidia-smi
!npm install -g localtunnel
import subprocess
import time
import sys
import os
import threading
import shutil
from google.colab import drive

    
if use_google_drive:
  drive.mount('/content/drive/')
  if not os.path.exists("/content/drive/MyDrive/TavernAI/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/")
  if not os.path.exists("/content/drive/MyDrive/TavernAI/characters/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/characters/")
  if not os.path.exists("/content/drive/MyDrive/TavernAI/chats/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/chats/")
else:
  if not os.path.exists("/content/drive"):
    os.mkdir("/content/drive")
  if not os.path.exists("/content/drive/MyDrive/"):
    os.mkdir("/content/drive/MyDrive/")

def copy_characters(use_google_drive=False):
  if not use_google_drive:
    return
  
  src_folder = "/TavernAI/public/characters"
  dst_folder = "/content/drive/MyDrive/TavernAI/characters"

  for filename in os.listdir(src_folder):
    src_file = os.path.join(src_folder, filename)
    dst_file = os.path.join(dst_folder, filename)

    if os.path.exists(dst_file):
      print(f"{dst_file} already exists. Skipping...")
      continue

    shutil.copy(src_file, dst_folder)
    print(f"{src_file} copied to {dst_folder}")


# 우가우가

!pip uninstall transformers
!pip install git+https://github.com/zphang/transformers.git@68d640f7c368bcaaaecfc678f11908ebbd3d6176

%cd /content
!git clone https://github.com/oobabooga/text-generation-webui

!mkdir text-generation-webui/logs

!ln -s text-generation-webui/logs .
!ln -s text-generation-webui/characters .
!ln -s text-generation-webui/models .
%cd text-generation-webui
!pip install -r requirements.txt

%cd /content/text-generation-webui/
!mkdir repositories
%cd /content/text-generation-webui/repositories/
!git clone https://github.com/qwopqwop200/GPTQ-for-LLaMa
%cd GPTQ-for-LLaMa
!python setup_cuda.py install
%cd /content/text-generation-webui/
!python download-model.py --text-only decapoda-research/llama-7b-hf
!wget -nc https://huggingface.co/decapoda-research/llama-7b-hf-int4/resolve/main/llama-7b-4bit.pt -P /content/models/llama-7b-hf/
os.remove("/content/models/llama-7b-hf/tokenizer_config.json")
!git clone https://github.com/peppertaco/token
shutil.copy("/content/text-generation-webui/token/tokenizer_config.json","/content/models/llama-7b-hf/tokenizer_config.json")
shutil.rmtree("/content/text-generation-webui/token")
!nohup python server.py --notebook --no-stream --share --model llama-7b-hf --gptq-bits 4 &

params = []

params.append('--cpu')

ExtrasModules = []

if (extras_enable_captioning):
  ExtrasModules.append('caption')

if (extras_enable_memory):
  ExtrasModules.append('summarize')

if (extras_enable_emotions):
  ExtrasModules.append('classify')

params.append(f'--classification-model={Emotions_Model}')
params.append(f'--summarization-model={Memory_Model}')
params.append(f'--captioning-model={Captions_Model}')
params.append(f'--enable-modules={",".join(ExtrasModules)}')

#TavernAI extras
%cd /
!git clone https://github.com/SillyLossy/TavernAI-extras
%cd TavernAI-extras
!pip install -r requirements.txt
!pip install tensorflow==2.11
cmd = f"python server.py {' '.join(params)}"
print(cmd)

extras_process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd='/TavernAI-extras', shell=True)
print('processId:', extras_process.pid)

while True:
    line = extras_process.stdout.readline().decode().strip()
    if "Running on " in line:
        break
    if not line:
        print('breaking on line')
        break
    print(line)

subprocess.call('nohup lt --port 5100 > ./extras.out 2> ./extras.err &', shell=True)
print('Waiting for lt init...')
time.sleep(5)

while True:
  if (os.path.getsize('./extras.out') > 0):
    with open('./extras.out', 'r') as f:
      lines = f.readlines()
      for x in range(len(lines)):
        if ('your url is: ' in lines[x]):
          print('TavernAI Extensions URL:')
          extras_url = lines[x].split('your url is: ')[1]
          print(extras_url)
      break
  if (os.path.getsize('./extras.err') > 0):
    with open('./extras.err', 'r') as f:
      print(f.readlines())
      break

#TavernAI
%cd /
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.37.2/install.sh | bash
!nvm install 19.1.0
!nvm use 19.1.0
!node -v
!git clone -b dev https://github.com/SillyLossy/TavernAI
copy_characters(use_google_drive)
%cd TavernAI
!npm install
time.sleep(1)
%env colab=2
%env colaburl=$url
if use_google_drive:
  %env googledrive=2
!nohup node server.js &
time.sleep(3)
print('###oobabooga API###')
lt_process = subprocess.Popen(['lt', '--port', '7860'], stdout=subprocess.PIPE)
line = lt_process.stdout.readline()
print(line.decode().strip())
print('### TavernAI Extensions LINK ###')
print(extras_url)
print('')
print('###TavernAI LINK###')
lt_process2 = subprocess.Popen(['lt', '--port', '8000'], stdout=subprocess.PIPE)
line = lt_process2.stdout.readline()
print(line.decode().strip())